In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)


In [2]:
''' 
Create NFL Production Score
'''

' \nCreate NFL Production Score\n'

In [3]:
measurables = pd.read_csv("Measurables.csv")
production = pd.read_csv("Production.csv")

In [4]:
trimmed_production = production.groupby("PlayerId", group_keys=False).apply(lambda g : g.sort_values("Season").head(4))
final_production = pd.DataFrame(index=trimmed_production.PlayerId.unique())

In [5]:
final_production['num_seasons'] = trimmed_production.groupby("PlayerId").Season.count()
# get all total values from production
final_production = pd.merge(final_production, trimmed_production.groupby("PlayerId").sum().drop("Season", axis=1), left_index=True, right_index=True)

In [6]:
final_production['GP%'] = final_production['GamesPlayed']/(final_production['num_seasons']*16)
final_production['GS%'] = final_production['GamesStarted']/(final_production['num_seasons']*16)
final_production['PosPlay%'] = final_production['PositivePlays']/final_production['Plays']
final_production['NegPlay%'] = final_production['NegativePlays']/final_production['Plays']
final_production['NeutPlay%'] = (final_production['Plays'] - final_production['PositivePlays'] - final_production['NegativePlays'])/final_production['Plays']

In [7]:
alpha_1 = 0.7
alpha_2 = 0.3
beta_1 = 1
beta_2 = 0.1
beta_3 = -1
w1=0.3
w2=0.7

final_production['Score'] = w1*(alpha_1*final_production['GP%'] + alpha_2*final_production['GS%']) + \
                            w2*(beta_1*final_production['PosPlay%'] + beta_2*final_production['NeutPlay%'] + beta_3*final_production['NegPlay%'])

In [8]:
'''
Clean/Engineer measurables/college data
'''

'\nClean/Engineer measurables/college data\n'

In [9]:
measurables.set_index("PlayerId",drop=True,inplace=True)

positions_to_remove = ['ST', 'RS', 'NT', 'LS', 'PK', 'PT']
cols_to_keep = ['ProPosition', 'Hgt', 'Wgt', 'Age', 'Forty', 'Arm', 'Hand', 'Wing', 'TenYard', 'TwentyYard',
                'ThreeCone', 'VJ', 'BJ', 'TwentyShuttle', 'SixtyShuttle', 'BP', 'Test_Acc%']

measurables['Test_Acc%'] = (measurables.Test/measurables.TestAttempt).round(4)

filtered_measurables = measurables.query("ProPosition not in @positions_to_remove")
filtered_measurables = filtered_measurables[cols_to_keep]


In [10]:
dataset = pd.merge(final_production[['Score']], filtered_measurables, how='left', right_index=True, left_index=True)

In [22]:
dataset = dataset[~dataset.ProPosition.isna()]

In [29]:
group_quantiles = dataset.groupby("ProPosition").Score.quantile(0.7)

In [30]:
def mark_success(group):
    percentile_threshold = group_quantiles[group.ProPosition]
    return (group['Score'] > percentile_threshold).astype(int)
    #return group

In [31]:
dataset['Success'] = dataset.apply(mark_success,axis=1)

/var/folders/z2/hw49lj911d1_nnsf962mcpr40000gn/T/ipykernel_5500/2313717056.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['Success'] = dataset.apply(mark_success,axis=1)


In [32]:
dataset

,Score,ProPosition,Hgt,Wgt,Age,Forty,Arm,Hand,Wing,TenYard,TwentyYard,ThreeCone,VJ,BJ,TwentyShuttle,SixtyShuttle,BP,Test_Acc%,Success
10018,0.264885,DT,74.88,293.0,23.4,4.79,31.500,9.375,76.375,1.68,2.79,7.23,29.0,9.02,4.37,NaN,29.0,0.5652,0
10034,0.126573,OT,79.63,308.0,22.2,5.10,33.000,10.625,80.125,1.78,2.95,7.62,30.0,9.05,4.68,NaN,33.0,0.5600,0
10081,0.353101,DT,75.88,336.0,21.3,5.28,33.125,10.125,79.625,1.90,3.03,7.62,24.5,NaN,4.82,NaN,19.0,0.9412,1
10085,0.172083,QB,78.88,242.0,21.5,4.92,33.875,9.875,79.375,1.73,2.87,NaN,NaN,NaN,NaN,NaN,NaN,0.7576,0
10100,0.163254,DE,75.38,284.0,21.7,4.76,35.500,9.625,80.375,1.74,2.77,7.16,34.5,9.10,4.40,NaN,25.0,0.5750,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99856,0.158457,WR,72.25,189.0,22.8,4.45,33.000,9.625,78.750,1.59,2.56,6.91,36.5,10.03,4.33,NaN,11.0,0.6774,0
99900,0.332741,RB,73.25,226.0,21.6,4.54,31.625,9.000,75.875,1.61,2.64,7.19,36.0,9.09,4.22,11.66,22.0,0.4595,0
99916,0.265140,TE,76.13,267.0,22.9,4.70,34.500,10.125,82.000,1.72,2.79,7.08,33.5,9.11,4.53,11.73,31.0,0.8333,1
99968,0.111176,ILB,74.00,230.0,22.3,4.67,31.625,9.375,74.750,1.62,2.67,7.01,29.0,9.07,4.34,12.00,22.0,0.6512,0


In [33]:
#dataset.to_csv("data/tree_data.csv")